In [1]:
#dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
from sqlalchemy import create_engine

In [2]:
#setup 2019 as current year
current_year = 2019

# #below code for getting current year in the datetime system
# current_date = datetime.datetime.today()
# current_year = current_date.year

In [3]:
#get header row for variable names
url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=2007"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
var_names = soup.find_all('th')

In [4]:
#create list of the variable names
var_name_list = []
for var in var_names:
    var_name_list.append(var.text)

In [5]:
#create empty lists for all variables
Date_List = []
Month1_List = []
Month2_List = []
Month3_List = []
Month6_List = []
Year1_List = []
Year2_List = []
Year3_List = []
Year5_List = []
Year7_List = []
Year10_List = []
Year20_List = []
Year30_List = []

In [6]:
#create list of years from pre-2008 U.S. economic recession to present year
range(2007, current_year)

year_list = []
for year in range(2007, current_year + 1):
    year_list.append(year)

In [7]:
#create list of urls for web scraping loop
url_list = []

for year in year_list:
    url = f"https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year={year}"
    url_list.append(url)

In [8]:
#loop through urls to scrape out data
for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #get all table rows
    results = soup.find('table', class_="t-chart").find_all("tr")
    
    #first row of results is the header
    #need to remove it to just get data rows
    results_noHeader = results[1:]
    
    #put all data rows into a list of separate data chunks
    #each chunk contains a data row of information
    data_list = []
    for index in range(len(results_noHeader)):
        data_chunk = results_noHeader[index].find_all('td')
        data_list.append(data_chunk)
    
    #loop through list of data to append data column values into appropriate variable list
    for data_col in data_list:
        
        #assign data columns to data variable names
        Date = data_col[0].text
        Month1 = data_col[1].text
        Month2 = data_col[2].text
        Month3 = data_col[3].text
        Month6 = data_col[4].text
        Year1 = data_col[5].text
        Year2 = data_col[6].text
        Year3 = data_col[7].text
        Year5 = data_col[8].text
        Year7 = data_col[9].text
        Year10 = data_col[10].text
        Year20 = data_col[11].text
        Year30 = data_col[12].text
        
        #append values to appropriate list       
        Date_List.append(Date)
        Month1_List.append(Month1)
        Month2_List.append(Month2)
        Month3_List.append(Month3)
        Month6_List.append(Month6)
        Year1_List.append(Year1)
        Year2_List.append(Year2)
        Year3_List.append(Year3)
        Year5_List.append(Year5)
        Year7_List.append(Year7)
        Year10_List.append(Year10)
        Year20_List.append(Year20)
        Year30_List.append(Year30)

In [9]:
date_list2 = []

for date in Date_List:
    date2 = pd.to_datetime(date, format='%m/%d/%y')
    date3 = str(date2)
    date4 = date3[:-9]
    date_list2.append(date4)

In [10]:
#append data into pandas dataframe
data_dict = {var_name_list[0] : date_list2,
            var_name_list[3] : Month3_List, var_name_list[4] : Month6_List, var_name_list[5] : Year1_List,
            var_name_list[6] : Year2_List, var_name_list[7] : Year3_List, var_name_list[8] : Year5_List,
            var_name_list[9] : Year7_List, var_name_list[10] : Year10_List}
Treasury_DF = pd.DataFrame(data = data_dict)

#remove all rows with missing data
Treasury_DF = Treasury_DF[Treasury_DF['3 mo'] != "\n\t\t\tN/A\n\t\t"]
Treasury_DF = Treasury_DF[Treasury_DF['6 mo'] != "\n\t\t\tN/A\n\t\t"]
Treasury_DF = Treasury_DF[Treasury_DF['1 yr'] != "\n\t\t\tN/A\n\t\t"]
Treasury_DF = Treasury_DF[Treasury_DF['2 yr'] != "\n\t\t\tN/A\n\t\t"]
Treasury_DF = Treasury_DF[Treasury_DF['3 yr'] != "\n\t\t\tN/A\n\t\t"]
Treasury_DF = Treasury_DF[Treasury_DF['5 yr'] != "\n\t\t\tN/A\n\t\t"]
Treasury_DF = Treasury_DF[Treasury_DF['7 yr'] != "\n\t\t\tN/A\n\t\t"]
Treasury_DF = Treasury_DF[Treasury_DF['10 yr'] != "\n\t\t\tN/A\n\t\t"]

In [11]:
#convert yield estimates to float
Treasury_DF = Treasury_DF.convert_objects(convert_numeric=True)

C:\Users\eyang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [12]:
#create connection to sqlite
engine = create_engine('sqlite:///db/BullsBearsDB.sqlite', echo=False)

In [13]:
#export scraped data to sqlite
Treasury_DF.to_sql('Treasury_Yield', con=engine, if_exists='replace', index=False)

In [14]:
#check if data was read into sqlite
engine.execute("SELECT * FROM Treasury_Yield").fetchall()

[('2007-01-02', 5.07, 5.11, 5.0, 4.8, 4.71, 4.68, 4.68, 4.68),
 ('2007-01-03', 5.05, 5.09, 4.98, 4.76, 4.69, 4.66, 4.66, 4.67),
 ('2007-01-04', 5.04, 5.07, 4.95, 4.71, 4.63, 4.61, 4.61, 4.62),
 ('2007-01-05', 5.05, 5.09, 4.98, 4.76, 4.68, 4.65, 4.65, 4.65),
 ('2007-01-08', 5.08, 5.13, 5.01, 4.78, 4.7, 4.66, 4.66, 4.66),
 ('2007-01-09', 5.08, 5.13, 5.02, 4.79, 4.7, 4.65, 4.65, 4.66),
 ('2007-01-10', 5.09, 5.13, 5.02, 4.81, 4.72, 4.68, 4.68, 4.69),
 ('2007-01-11', 5.11, 5.15, 5.05, 4.86, 4.78, 4.73, 4.73, 4.74),
 ('2007-01-12', 5.09, 5.15, 5.06, 4.88, 4.81, 4.76, 4.76, 4.77),
 ('2007-01-16', 5.11, 5.16, 5.06, 4.86, 4.79, 4.74, 4.74, 4.75),
 ('2007-01-17', 5.12, 5.16, 5.08, 4.91, 4.83, 4.78, 4.78, 4.79),
 ('2007-01-18', 5.12, 5.16, 5.07, 4.89, 4.8, 4.75, 4.75, 4.75),
 ('2007-01-19', 5.14, 5.17, 5.09, 4.93, 4.83, 4.78, 4.78, 4.78),
 ('2007-01-22', 5.13, 5.18, 5.09, 4.91, 4.82, 4.77, 4.76, 4.76),
 ('2007-01-23', 5.14, 5.18, 5.1, 4.94, 4.85, 4.81, 4.81, 4.81),
 ('2007-01-24', 5.13, 5.17, 5.0

In [15]:
# #export dataset to csv file
Treasury_DF.to_csv("datasets/Treasury_Yield.csv", index=False, header=True)